In [48]:
import timm
import torch
from torchvision.transforms import Compose, Resize, ToTensor
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [49]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [51]:
transform = Compose([Resize((224, 224)), ToTensor()])
dataset = ImageFolder(root='../data/validation', transform=transform)
data_loader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4)

In [55]:
def load_model(model_name):
    model = timm.create_model(model_name, pretrained=True)
    model.to(device)
    model.eval()
    return model

In [54]:
swin_b = load_model('swin_base_patch4_window7_224')
swin_s = load_model('swin_small_patch4_window7_224')
swin_t = load_model('swin_tiny_patch4_window7_224')
swin_l = load_model('swin_large_patch4_window7_224')

model.safetensors:   0%|          | 0.00/200M [00:00<?, ?B/s]

C:\Users\Weiye\anaconda3\envs\dsa5204-project\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Weiye\.cache\huggingface\hub\models--timm--swin_small_patch4_window7_224.ms_in22k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

C:\Users\Weiye\anaconda3\envs\dsa5204-project\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Weiye\.cache\huggingface\hub\models--timm--swin_tiny_patch4_window7_224.ms_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/788M [00:00<?, ?B/s]

C:\Users\Weiye\anaconda3\envs\dsa5204-project\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Weiye\.cache\huggingface\hub\models--timm--swin_large_patch4_window7_224.ms_in22k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [60]:
def compute_top_k_acc(pred_labels, true_labels, k):
    _, top_k_pred = pred_labels.topk(k, dim=1)
    correct_top_k = (top_k_pred == true_labels.view(-1, 1)).sum().item()
    return correct_top_k

In [61]:
model_results = {
    'swin_b': [0, 0],
    'swin_s': [0, 0],
    'swin_t': [0, 0],
    'swin_l': [0, 0],
}

total = 0

In [66]:
with torch.no_grad():
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        
        swin_b_out = swin_b(images)
        model_results['swin_b'][0] += compute_top_k_acc(swin_b_out, labels, 1)
        model_results['swin_b'][1] += compute_top_k_acc(swin_b_out, labels, 5)
        
        swin_s_out = swin_s(images)
        model_results['swin_s'][0] += compute_top_k_acc(swin_s_out, labels, 1)
        model_results['swin_s'][1] += compute_top_k_acc(swin_s_out, labels, 5)
        
        swin_t_out = swin_t(images)
        model_results['swin_t'][0] += compute_top_k_acc(swin_t_out, labels, 1)
        model_results['swin_t'][1] += compute_top_k_acc(swin_t_out, labels, 5)
        
        swin_l_out = swin_l(images)
        model_results['swin_l'][0] += compute_top_k_acc(swin_l_out, labels, 1)
        model_results['swin_l'][1] += compute_top_k_acc(swin_l_out, labels, 5)
        
        total += labels.size(0)
        if total % 100 == 0:
            print(f'Evaluated {total} images...')

Evaluated 1600 images...
Evaluated 3200 images...
Evaluated 4800 images...
Evaluated 6400 images...
Evaluated 8000 images...
Evaluated 9600 images...
Evaluated 11200 images...
Evaluated 12800 images...
Evaluated 14400 images...
Evaluated 16000 images...
Evaluated 17600 images...
Evaluated 19200 images...
Evaluated 20800 images...
Evaluated 22400 images...
Evaluated 24000 images...
Evaluated 25600 images...
Evaluated 27200 images...
Evaluated 28800 images...
Evaluated 30400 images...
Evaluated 32000 images...
Evaluated 33600 images...
Evaluated 35200 images...
Evaluated 36800 images...
Evaluated 38400 images...
Evaluated 40000 images...
Evaluated 41600 images...
Evaluated 43200 images...
Evaluated 44800 images...
Evaluated 46400 images...
Evaluated 48000 images...
Evaluated 49600 images...
Evaluated 50000 images...


In [67]:
for name, model_result in model_results.items():
    top1_acc = 100 * model_result[0] / total
    top5_acc = 100 * model_result[1] / total
    print(f"Model: {name} - Top-1 Accuracy: {top1_acc:.2f}%, Top-5 Accuracy: {top5_acc:.2f}%")

Model: swin_b - Top-1 Accuracy: 81.14%, Top-5 Accuracy: 94.63%
Model: swin_s - Top-1 Accuracy: 78.90%, Top-5 Accuracy: 94.95%
Model: swin_t - Top-1 Accuracy: 76.66%, Top-5 Accuracy: 93.21%
Model: swin_l - Top-1 Accuracy: 82.64%, Top-5 Accuracy: 95.29%
